In [ ]:
import json
import pandas as pd

Loading and processing the dataset

In [ ]:
def load_data(csv_path, nrows=None):
    """
    Load and preprocess the Google Analytics customer revenue dataset.
    
    Parameters:
        csv_path (str): Path to the CSV file.
        nrows (int, optional): Number of rows to load for testing purposes. Loads all rows if None.
        
    Returns:
        pd.DataFrame: Processed DataFrame with flattened JSON columns and formatted data.
    """
    
    # Define columns that contain JSON data
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    # Load the CSV file with JSON columns parsed and fullVisitorId as a string
    df = pd.read_csv(
        csv_path,
        converters={column: json.loads for column in JSON_COLUMNS},  # Convert JSON columns to dictionaries
        dtype={'fullVisitorId': 'str'},  # Ensure fullVisitorId is loaded as a string for uniqueness
        nrows=nrows  # Load specified number of rows if provided
    )
    
    # Flatten each JSON column and merge back into the main DataFrame
    for column in JSON_COLUMNS:
        # Convert JSON data in the column to a DataFrame
        column_as_df = pd.json_normalize(df[column])
        
        # Rename the columns to indicate their source (e.g., device_isMobile)
        column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
        
        # Drop the original JSON column and merge the new flattened columns
        df = df.drop(column, axis=1).join(column_as_df)
    
    # Drop columns that are either not useful or too complex for initial analysis
    df.drop(columns=['customDimensions', 'hits'], inplace=True)
    
    # Fill missing values for key numerical columns with 0
    # - totals_pageviews: Number of pages viewed in a session, can be 0 if no pages were viewed
    # - totals_transactionRevenue: Revenue generated, fill missing with 0 as no revenue
    df.fillna({
        'totals_pageviews': 0,
        'totals_transactionRevenue': 0
    }, inplace=True)
    
    # Convert specific columns to integers for consistency
    df['device_isMobile'] = df['device_isMobile'].astype(int)  # Convert isMobile to 1 or 0
    df['totals_transactionRevenue'] = df['totals_transactionRevenue'].astype(int)  # Revenue as integer
    
    # Ensure all object (string) columns are consistently treated as strings
    cat_cols = df.select_dtypes(include='object').columns
    df[cat_cols] = df[cat_cols].astype(str)
    
    return df